## Web Scraping for Septic Permits of Watauga County, NC

In [107]:
# Start with the usual imports
# We'll use these throughout
import pandas as pd
from bs4 import BeautifulSoup
import requests
# Import the webdriver from selenium
from selenium import webdriver

In [281]:
driver = webdriver.Chrome()

### 1. Navigate to URL

In [282]:
# Open the URL
url = "https://public.cdpehs.com/NCENVPBLo/OSW_PROPERTY/ShowOSW_PROPERTYTablePage.aspx?ESTTST_CTY=D5"
driver.get(url)

### 2. Select the "NEW" type septic permits

In [283]:
from selenium.webdriver.support.ui import Select

# Assuming you have a WebDriver instance named 'driver'

# Find the dropdown element using the class selector
dropdown_element = driver.find_element(By.ID, "ctl00_PageContent_APPL_TYPE_IDFilter1")

# Use the Select class for interacting with dropdowns
dropdown = Select(dropdown_element)

# Select the 'NEW' option by its visible text
dropdown.select_by_visible_text('NEW')

# Perform other actions or continue with your web scraping logic

### 3. Select the date range

In [284]:
# Find start date input element
start_date_input = driver.find_element(By.ID, "ctl00_PageContent_APPL_DATEFromFilter1")

# Clear the input
start_date_input.clear()

# Set the start date
start_date_input.send_keys("1/1/2022")

In [285]:
# Find start date input element
end_date_input = driver.find_element(By.ID, "ctl00_PageContent_APPL_DATEToFilter1")

# Clear the input
end_date_input.clear()

# Set the end date
end_date_input.send_keys("12/31/2022")

### 4. Select the search button and click it

In [286]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Assuming you have a WebDriver instance named 'driver'

# Find the button using its id
button = driver.find_element(By.ID, "ctl00_PageContent_OSW_PROPERTYSearchButton__Button")

# Click on the button
button.click()

### 5. Set the row number on each page to "100"

In [287]:
from selenium.webdriver.common.keys import Keys

# Find row number input element
row_n_input = driver.find_element(By.ID, "ctl00_PageContent_OSW_PROPERTYPagination__PageSize")

# Clear the input
row_n_input.clear()

# Set the row number to 100
row_n_input.send_keys("100")

# Simulate pressing the Enter key
row_n_input.send_keys(Keys.ENTER)

### 5. Use BeautifulSoup to parse the results

In [288]:
from selenium.common.exceptions import NoSuchElementException
import time

# Initialize an empty list to store scraped data
scraped_data = []

# Initialize a variable to store the previous page content
prev_page_content = None

# Assuming there is a 'Next' button for pagination
while True:
    # Add your scraping logic here using BeautifulSoup
    propertySoup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # Check if the current page content is the same as the previous page
    current_page_content = str(propertySoup)
    if current_page_content == prev_page_content:
        # If the content is the same, break the loop as you've reached the end
        break
    
    # Update the previous page content
    prev_page_content = current_page_content
    
    # Extract and process data from propertySoup
    # Select the table
    table = propertySoup.select_one(table_selector)

    # Select all row (<tr>) elements in the table tbody
    rows = table.select("tbody tr:not(.tch)")

    # Add your logic to extract and process data from each row
    for row in rows:
        row_data = [str(content).strip() for cell in row.select("td.ttc") for content in cell.contents if content.name is None]
        scraped_data.append(row_data)
        
    # Click the 'Next' button to navigate to the next page
    try:
        next_button = driver.find_element(By.ID, 'ctl00_PageContent_OSW_PROPERTYPagination__NextPage')
        next_button.click()
        # Wait for some time to allow the page to load (adjust as needed)
        time.sleep(2)
    except NoSuchElementException:
        # Break the loop if there is no 'Next' button
        break

In [289]:
len(scraped_data)

1088

In [290]:
permits = pd.DataFrame(
    scraped_data,
    columns=[
        "MISSED",
        "PIN",
        "Permit #",
        "Street Address",
        "City, State, ZIP Code",
        "Owner Name",
        "Applicant Name",
        "unknown",
        "Block Lot",
        "Subdivision",
    ],
)

permits.head()

,MISSED,PIN,Permit #,Street Address,"City, State, ZIP Code",Owner Name,Applicant Name,unknown,Block Lot,Subdivision
0,,13190071059,53795,1067 Winding Ridge Drive,"West Jefferson,\n North Carolina\n 28694","McNeil, Carrie / Moss, Sarah","McNeil, Carrie / Moss, Sarah",,59,Fairway Ridge Mountain Aire (0.828 acre)
1,,2859-41-2337,65-83G,1007 Laurel Circle East,"Deep Gap,\n North Carolina\n 28618","Sims, Anna Marie","Sims, Anna Marie",,LR-22R,Powder Horn Mountain
2,,1980-33-9058,9510549,1898 Broadstone Rd.,"Banner Elk,\n North Carolina\n 28604","Read, Mark","Read, Mark",,,
3,,2920-36-7670,9510771,Bamboo Road,"Boone,\n North Carolina\n 28607","Phillips, Donovan","Phillips, Donovan",,,Greenbriar
4,,18878-79-3126,9511467,Field Stone,",\n North Carolina","Corey, Shawn","Corey, Shawn",,9,Seven Devils


In [291]:
# Split the column into three columns
permits[['City', 'State', 'ZIP Code']] = permits['City, State, ZIP Code'].str.split('\n', expand=True)

# Clean up the values by stripping extra spaces
permits['City'] = permits['City'].str.replace(',', '')
permits['State'] = permits['State'].str.strip()
permits['ZIP Code'] = permits['ZIP Code'].str.strip()

permits.head()

,MISSED,PIN,Permit #,Street Address,"City, State, ZIP Code",Owner Name,Applicant Name,unknown,Block Lot,Subdivision,City,State,ZIP Code
0,,13190071059,53795,1067 Winding Ridge Drive,"West Jefferson,\n North Carolina\n 28694","McNeil, Carrie / Moss, Sarah","McNeil, Carrie / Moss, Sarah",,59,Fairway Ridge Mountain Aire (0.828 acre),West Jefferson,North Carolina,28694
1,,2859-41-2337,65-83G,1007 Laurel Circle East,"Deep Gap,\n North Carolina\n 28618","Sims, Anna Marie","Sims, Anna Marie",,LR-22R,Powder Horn Mountain,Deep Gap,North Carolina,28618
2,,1980-33-9058,9510549,1898 Broadstone Rd.,"Banner Elk,\n North Carolina\n 28604","Read, Mark","Read, Mark",,,,Banner Elk,North Carolina,28604
3,,2920-36-7670,9510771,Bamboo Road,"Boone,\n North Carolina\n 28607","Phillips, Donovan","Phillips, Donovan",,,Greenbriar,Boone,North Carolina,28607
4,,18878-79-3126,9511467,Field Stone,",\n North Carolina","Corey, Shawn","Corey, Shawn",,9,Seven Devils,,North Carolina,None


In [292]:
# Drop the unnecessary columns
columns_to_drop = ['MISSED', 'City, State, ZIP Code', 'unknown']
permits_clean = permits.drop(columns=columns_to_drop)
permits_clean

,PIN,Permit #,Street Address,Owner Name,Applicant Name,Block Lot,Subdivision,City,State,ZIP Code
0,13190071059,53795,1067 Winding Ridge Drive,"McNeil, Carrie / Moss, Sarah","McNeil, Carrie / Moss, Sarah",59,Fairway Ridge Mountain Aire (0.828 acre),West Jefferson,North Carolina,28694
1,2859-41-2337,65-83G,1007 Laurel Circle East,"Sims, Anna Marie","Sims, Anna Marie",LR-22R,Powder Horn Mountain,Deep Gap,North Carolina,28618
2,1980-33-9058,9510549,1898 Broadstone Rd.,"Read, Mark","Read, Mark",,,Banner Elk,North Carolina,28604
3,2920-36-7670,9510771,Bamboo Road,"Phillips, Donovan","Phillips, Donovan",,Greenbriar,Boone,North Carolina,28607
4,18878-79-3126,9511467,Field Stone,"Corey, Shawn","Corey, Shawn",9,Seven Devils,,North Carolina,None
...,...,...,...,...,...,...,...,...,...,...
1083,1973-88-0959,,1325 Laurel Branch Road,"Balduf, Jeannie/Balduf, David","Balduf, Jeannie/Balduf, David",,,Vilas,North Carolina,28692
1084,1973-88-0959,,1325 Laurel Branch Road,"Balduf, Jeannie/Balduf, David","Balduf, Jeannie/Balduf, David",,,Vilas,North Carolina,28692
1085,02182085,,Teaberry Road,"Howell, Bobby Avery (OP Hold)","Howell, Bobby Avery",,,Warrensville,North Carolina,28693
1086,1878-47-1979,,Telemark Ln,"Sherman, Jason","Sherman, Jason",41,Maple Springs,Seven Devils,North Carolina,28607


In [293]:
permits_clean.to_csv('new_permits_2022.csv', index=False)